# Genomic DB Scratch Pad
Interactive session for loading data and querying with SQL.

In [8]:
import sys
sys.path.insert(0, "../src")

from sqlalchemy import text
from genomic_db.db import get_session, engine
from genomic_db.models import Base, Genome, Chromosome, Gene

## Reset (truncate all tables)

In [5]:
with engine.connect() as conn:
    conn.execute(text("TRUNCATE genes, chromosomes, genomes CASCADE"))
    conn.commit()
    print("All tables truncated")

All tables truncated


## Load sample data

In [5]:
session = get_session()

# Skip if sample data already exists
existing = session.query(Genome).filter_by(name="Example Genome").first()
if existing:
    print(f"Sample data already exists: {existing.name}")
else:
    genome = Genome(name="Example Genome", description="A minimal example genome for testing")
    session.add(genome)
    session.flush()

    chr1 = Chromosome(name="chr1", length=1000000, genome_id=genome.id)
    chr2 = Chromosome(name="chr2", length=800000, genome_id=genome.id)
    session.add_all([chr1, chr2])
    session.flush()

    genes = [
        Gene(name="GENE_A", start_position=1000, end_position=2500, strand="+", sequence="ATGCGTACGATCGATCGATCG", chromosome_id=chr1.id),
        Gene(name="GENE_B", start_position=5000, end_position=7000, strand="-", sequence="GCTAGCTAGCTAGCTAGCTA", chromosome_id=chr1.id),
        Gene(name="GENE_C", start_position=100, end_position=500, strand="+", sequence="TTAACCGGTTAACCGGTTAA", chromosome_id=chr2.id),
    ]
    session.add_all(genes)
    session.commit()

    print(f"Loaded: {genome.name} with {len(genome.chromosomes)} chromosomes")
session.close()

Loaded: Example Genome with 2 chromosomes


## Query with raw SQL

In [10]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT * FROM genomes"))
    for row in result:
        print(row)

(6, 'Example Genome', 'A minimal example genome for testing', datetime.datetime(2026, 2, 14, 17, 33, 34, 943856, tzinfo=datetime.timezone.utc), datetime.datetime(2026, 2, 14, 17, 33, 34, 943856, tzinfo=datetime.timezone.utc))


In [11]:
with engine.connect() as conn:
    result = conn.execute(text("""
        SELECT g.name AS gene, c.name AS chromosome, gn.name AS genome
        FROM genes g
        JOIN chromosomes c ON g.chromosome_id = c.id
        JOIN genomes gn ON c.genome_id = gn.id
    """))
    for row in result:
        print(row)

('GENE_B', 'chr1', 'Example Genome')
('GENE_A', 'chr1', 'Example Genome')
('GENE_C', 'chr2', 'Example Genome')


In [6]:
# Write your own SQL here
with engine.connect() as conn:
    result = conn.execute(text("SELECT * from chromosomes"))
    for row in result:
        print(row)